计算机视觉的多种用途：

abcd分别是从最简单的图片分类，到对象识别（定位），图片（语义）分割，到物体识别与分割。

具体实际应用太多。

早期图像识别，需使用专家知识进行特征提取。

 例如色彩、形状、纹理等直方图组成的特征。

神经网络广泛应用后，只要喂数据训练就可以自动提取特征。

处理二维图像，使用卷积神经网络有很多好处：

1. 权值共享（卷积核移动卷积的操作，体现了权值共享，因为一张图只用一个卷积核就能提取出一张特征图）

2. 保留位置相关信息，全连层网络把像素（数值）打乱了，位置信息丢了。

 卷积神经网络知识点：卷积层、池化层、激活函数、批归一化操作、Dropout增加鲁棒性。

1. 卷积

这里是二维卷积，就是滑动窗口计算点积。卷积会造成输出特征图小于输入图像，因此为了输入输出大小一致，会给输入图像进行边缘补充。

补0或者对称填数什么的，对应着不同的卷积方法。

步长：就是卷积核每次向右移动的格子数。

 2. 池化

就是把小窗口变成一个单一数据，来平滑异常值。

3. 多路输入、多路输出

意思就是RGB三通道，得准备三组卷积核。

4. 批归一化

这段概念直接抄来：

通常我们会对神经网络的数据进行标准化处理，处理后的样本数据集满足均值为0，方差为1的统计分布，这是因为当输入数据的分布比较固定时，有利于算法的稳定和收敛。对于深度神经网络来说，由于参数是不断更新的，即使输入数据已经做过标准化处理，但是对于比较靠后的那些层，其接收到的输入仍然是剧烈变化的，通常会导致数值不稳定，模型很难收敛。BatchNorm能够使神经网络中间层的输出变得更加稳定，并有如下三个优点：

使学习快速进行（能够使用较大的学习率）

降低模型对初始值的敏感性

从一定程度上抑制过拟合

BatchNorm主要思路是在训练时按mini-batch为单位，对神经元的数值进行归一化，使数据的分布满足均值为0，方差为1。

预测时：使用训练过程中保存的训练样本的均值方差给预测样本做归一化。

 5. Dropout

这段直接抄过来：

丢弃法（Dropout）是深度学习中一种常用的抑制过拟合的方法，其做法是在神经网络学习过程中，随机删除一部分神经元。训练时，随机选出一部分神经元，将其输出设置为0，这些神经元将不对外传递信号。

图14 是Dropout示意图，左边是完整的神经网络，右边是应用了Dropout之后的网络结构。应用Dropout之后，会将标了X的神经元从网络中删除，让它们不向后面的层传递信号。在学习过程中，丢弃哪些神经元是随机决定，因此模型不会过度依赖某些神经元，能一定程度上抑制过拟合。


在预测场景时，会向前传递所有神经元的信号，可能会引出一个新的问题：训练时由于部分神经元被随机丢弃了，输出数据的总大小会变小。比如：计算其L1L1L1范数会比不使用Dropout时变小，但是预测时却没有丢弃神经元，这将导致训练和预测时数据的分布不一样。为了解决这个问题，飞桨支持如下两种方法：

downgrade_in_infer
训练时以比例rrr随机丢弃一部分神经元，不向后传递它们的信号；预测时向后传递所有神经元的信号，但是将每个神经元上的数值乘以 (1?r)(1 - r)(1?r)。

upscale_in_train
训练时以比例rrr随机丢弃一部分神经元，不向后传递它们的信号，但是将那些被保留的神经元上的数值除以 (1?r)(1 - r)(1?r)；预测时向后传递所有神经元的信号，不做任何处理。

在飞桨dropout API中，paddle.fluid.layers.dropout通过dropout_implementation参数来指定用哪种方式对神经元进行操作，dropout_implementation参数的可选值是'downgrade_in_infer'或'upscale_in_train'，缺省值是'downgrade_in_infer'。